# Newton-type Optimization: Unconstrained Optimization
Author: Suhan Shety, suhan.n.shetty@gmail.com


This notebook implements several Newton-type optimization algorithms using tensorflow 2.0. They are iterative in nature.

Suppose we have,

Forward model:  $\mathit{M}(x): \mathbb{R}^m \rightarrow \mathbb{R}^n$, 

Target vector:   $Y \in \mathbb{R}^n $,

The fit error vector:  $\mathit{F}(x) = Y-\mathit{M}(x)$,

Squared Error:  $\mathit{f}(x) = ||\mathit{F}(x)|| $


An update in the optimization parameter in the Newton-like method is given by: $$x_{k+1} = x_{k} - B_{k}^{-1} \nabla f(x_k)$$

The Newton-type methods differ by approximations to the Hessian matrix $B$.

**Exact Newton(EN)**: uses the exact Hessian. So, its expensive computationally,
        $$B_{k} = \nabla ^{2} f(x_k)$$

**Gradient Descent(GD)**:
        $$B_{k} = \alpha \mathit{I},$$ $$ \text{where $\alpha$ is learning rate} $$


**Gauss-Newton(GN)**:
         $$B = J^{T}J$$
         $$\text{where $J = \mathit{D} F(x_k)$ is the Jacobian matrix}$$
         
**Levenberg-Larquardt(LM)**: $$ B = J^{T}J + \alpha \mathit{I} $$
      where $J = \mathit{D} F(x_k)$  is the Jacobian matrix, 
         and $\alpha$ is a regularizer. $\alpha=0$ results in GN method and $
         \alpha >> 1$ results in GD
         
**BFGS:**   Approximates the hessians using a history of gradients
    $$B_{k+1} = B_k - \frac{s_k'B_k'B_k s_k}{s_k' B_k s} + \frac{y_k y_k'}{s_k'y_k}$$
    
where $s_k = x_{k+1}-x_{k}$, and $y_k = \nabla f(x_{k+1})-\nabla f(x_{k})$


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Generate data and choose a model
N = 200 # Number of data points
m = 20 # number of parameters (to learn)
n = 3 # input dimension (state)

# Model parameters (actual)
x0 = tf.random.uniform((m,1),minval=0, maxval=1)

# Model Input (state)
Theta = tf.random.uniform((N,n),minval=0, maxval=1) 
t0 = tf.reshape(Theta[:,0],shape=(N,1))
t1 = tf.reshape(Theta[:,1],shape=(N,1))
t2 = tf.reshape(Theta[:,2],shape=(N,1))


# Model output
x20 = tf.reduce_mean(x0*x0)
x10 = tf.reduce_mean(x0)
x00 = 1.
Z0 = x20*t2 + x10*t1 + x00*t0    
Y = Z0 + 0.001*tf.random.normal((N,1)) #target vector/ Data Outputs with some noise


In [13]:
# Netwon's type methods

class newton_opt:
    """
    x0: intial param
    fcn: returns error vector (fitting error )
    alpha: learning rate or regularizer
    method: GD(Gradient Descent),
            GN(Gauss Newton),
            LM(Levenberg-Marquardt),
            EN(Exact NEwton),
            BFGS  
    global_: if True, uses line-search based on Armijo-Backtracking algorithm
    """
    
    def __init__(self, x0, fcn, method="GD", global_=False, alpha=0.0001, eps=1E-3):
        self.eps = eps
        self.x = tf.Variable(tf.reshape(x0,(-1,1)))
        self.m = tf.shape(x0).numpy()[0]
        self.f = fcn
        self.alpha = alpha
        self.method = method # Optimization Method
        self.global_ = global_
        if self.method in ["GN"]:
            self.alpha = 0. # GN is a special case of LM with alpha=0
        if self.method in ["GD","EN","BFGS"]:
            self.f = self.f_sq_err(fcn) #Use the least square cost directly
        
        self.fs = self.f_sq_err(fcn)
            
       
    def jacob(self, x):
        """Returns the Jacobian of the objective function"""
        with tf.GradientTape() as t:
            y = self.f(x)
        dy_dx = t.jacobian(y,x)
        return tf.reshape(dy_dx,(-1,self.m))
 
    def f_sq_err(self,fcn):
        """Return a function which gives norm of error vector"""
        def f_sq(x=self.x):
            err_ = fcn(x)
            output = tf.reduce_sum(err_*err_)
            return output
        return f_sq

    def hess(self, x):
        """Returns the Hessian of scalar function """
        with tf.GradientTape() as t:
            with tf.GradientTape() as t2:
                y = self.f(x)
                assert not (tf.size(y)>1), "The obj fcn has to be scalar valued!!"
            dy_dx = t2.jacobian(y, x)
        d2y_dx2 = t.jacobian(dy_dx, x)
        return tf.reshape(d2y_dx2,(self.m,self.m))
    
    
    def optimize(self, max_it=100):
        i = 0
        B = 1*tf.eye(self.m) #intialization for BFGS
        while True: 
            if self.method in ["GD","EN","BFGS"]:
                gradf = tf.transpose(self.jacob(self.x))
            elif self.method in ["GN","LM"]:
                F = self.f(self.x)
                J = self.jacob(self.x) # shape=(N,m)
                Jt = tf.transpose(J)
                gradf = tf.matmul(Jt,F)

            
            if self.method =="GD":
                B = (1/self.alpha)*tf.eye(self.m)

            elif self.method in ["GN", "LM"]:
                B = tf.matmul(Jt,J)+self.alpha*tf.eye(self.m)
            
            elif self.method=="EN":
                B = self.hess(self.x)    

            invB = tf.linalg.pinv(B)
            
            # Step size 
            dx = -tf.matmul(invB,gradf)
            
            # Line search based on Armijo condition and backtracking
            if self.global_ == True:
                gamma_ = 0.1 # choose in (0,1)
                beta_ = 0.8 # choose in (0,1)
                t = 1. #
                while self.fs(self.x+t*dx) >= (self.fs(self.x)+gamma_*t*tf.reduce_sum(tf.multiply(gradf, dx))) or t < 0.1:
                    t=beta_*t
                dx = t*dx
                
                
            

            # Update
            xold = tf.Variable(self.x)
            self.x = tf.Variable(self.x+dx)
            
            if self.method == "BFGS":
                s = dx
                st = tf.transpose(s)
                yt = self.jacob(self.x)-self.jacob(xold)
                y = tf.transpose(yt)
                c1 = tf.matmul(tf.matmul(st,B),s)
                c2 = tf.matmul(st,y)
                Bs = tf.matmul(B,s)
                B = B - (1/c1)*tf.matmul(Bs,tf.transpose(Bs)) + (1/c2)*tf.matmul(y,yt)
             
            # Exit condition
            i = i+1

            if np.linalg.norm(dx.numpy())<self.eps or i>max_it :
                err_ = self.f(self.x).numpy().reshape(-1,)
                print("Converges in iteraton: ",i)
                print("Fitting Error: ", np.dot(err_,err_)**(0.5))
                if i>max_it :
                    print("Warning: Maximum iteration crossed. ", 
                          "Algorithm may not have converged. Increase max_it")
                break

        return self.x


In [3]:
def f_vec_err(x):
    """
    Returns the error vector from the forward map/model.
    Can use any nonlinear function.
    
    params:
        x: the optimization parameter 
        return:  error vector=(Y-model(x))
        
    """
    x2_ = tf.reduce_mean(x*x)
    x1_ = tf.reduce_mean(x)
    x0_ =1
    Z = x2_*t2+x1_*t1+x0_*t0  
    err_ = Y-Z
    output =  tf.reshape(err_,(N,1))
    
    return output
 
# def f_sq_err(x, fcn=f_vec_err):
#     """
#     Define the forward map of the objective function.
#     Can use any nonlinear function
    
#     params:
#         fcn: Returns error vector for the current estimate
#         x: the optimization parameter
#     """
#     err_ = fcn(x)

#     output = tf.reduce_sum(err_*err_)
        
#     return output



In [14]:
# Initialization of the parameter x 
x0_ = tf.Variable(tf.random.uniform((m,1),minval=-1, maxval=1)) 
eps_ = 1E-5

In [15]:
opt = newton_opt(x0=x0_, fcn=f_vec_err, method="GN", eps=eps_)
x = opt.optimize(max_it=100)

Converges in iteraton:  6
Fitting Error:  0.013295533978667616


In [16]:
opt = newton_opt(x0=x0_, fcn=f_vec_err, method="LM", alpha= 0.1, eps=eps_)
x = opt.optimize(max_it=100)

Converges in iteraton:  16
Fitting Error:  0.013295434926366598


In [17]:
opt = newton_opt(x0=x0_, fcn=f_vec_err, method="BFGS", eps=eps_)
x = opt.optimize(max_it=100)

Converges in iteraton:  43
Fitting Error:  0.00017677122855504532


In [18]:
opt = newton_opt(x0=x0_, fcn=f_vec_err, method="EN", eps=eps_)
x = opt.optimize(max_it=100)

Converges in iteraton:  11
Fitting Error:  0.00017677122855504532


In [19]:
opt = newton_opt(x0=x0_, fcn=f_vec_err, method="GD", alpha=0.01, eps=eps_)
x = opt.optimize(max_it=100)

Converges in iteraton:  101
Fitting Error:  0.307903652927043
